# The Battle of the Neighborhoods - Week 1
## Introduction & Business Problem :
### Problem Background:
Dubai is a large cosmopolitan city with lots to do. It holds a lot of events to keep attracting locals - both citizens and expats - as well as tourists.

### Problem Description:
As a tourist, you want to see where most of these events are concetrated so you can book your hotel accordingly to minimize travel and save on taxi fare.


### Target Audience:
Anyone who is interested in various events in Dubai.


The date set is used from 

https://www.dubaipulse.gov.ae/data/cda-events/cda_volunteers_activities_and_events-open?organisation=cda&service=cda-events

In [6]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Abdul\AppData\Local\Continuum\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.7 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0
  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi              pkgs

In [2]:
df=pd.read_csv('Volunteers_Activities_And_Events.csv')

In [3]:
df.head()

,eventenglishtitle,eventarabictitle,eventarabicorganizername,eventenglishorganizername,eventstartingdate,eventendingdate,introductiondate,locationinarabic,locationinenglish,eventlogo,eventcategory,maplocationlongitude,maplocationlatitude,createddate,typeofevent
0,حملة جمعية دار البر برمضان والعيدين,Campaign of DAR Al Bar society in Ramadan and Eid,جمعية دار البر,DAR Al Bar society,03/01/2018 00:00:00,07/31/2018 00:00:00,02/28/2018 00:00:00,جمعية دار البر,DAR Al Bar society,NaN,NaN,NaN,NaN,NaN,الاعمال الخيرية
1,البرنامج التطوعي لموظفي جمعية دار البر,Volunteer Program for the staff of DAR Al BER ...,جمعية دار البر,DAR Al BER society,02/01/2018 00:00:00,12/01/2018 00:00:00,01/31/2018 00:00:00,جمعية دار البر,DAR Al BER society,NaN,NaN,NaN,NaN,NaN,آخر
2,Graphic Design Volunteers,متطوعين في تخصص Graphic Design للعمل على تصمي...,مؤسسة دبي لرعاية النساء والأطفال,Dubai Foundation for Women and Children,04/15/2018 00:00:00,05/31/2018 00:00:00,04/12/2018 00:00:00,مؤسسة دبي لرعاية النساء والأطفال,Dubai Foundation for Women and Children,NaN,NaN,55.508358,25.156502,NaN,الفنون والثقافة
3,Volunteer for Punishing & Correctional Institu...,متطوع للمؤسسات العقابية و الاصلاحية,ادارة الفئات الاكثر عرضة للضرر,Vulnerable Group department,04/01/2018 00:00:00,05/31/2018 00:00:00,04/01/2018 00:00:00,مبنى المؤسسات العقابية و الاصلاحية,HQ Punishing & Correctional Institutions,NaN,NaN,55.508208,25.151772,NaN,حملات توعية
4,Administrative organization,تنظيم إداري,مكتب الشؤون القانونية,legal office,04/20/2018 00:00:00,06/19/2018 00:00:00,04/20/2018 00:00:00,مكتب الشؤون القانونية بهيئة تنمية المجتمع - مب...,legal office - cda,NaN,NaN,55.272253,25.240559,NaN,إجتماعي


In [15]:
df.dropna(subset = ["maplocationlongitude"], inplace=True)

In [5]:
CLIENT_ID = '3TTX1OYJ5M1TNYZNOFUQIYCQWE4OL2J2ZKX1LQNSJEUD2SRA' # your Foursquare ID
CLIENT_SECRET = 'U5TUV4RLEPPEFPEIO32K3CB5LPLW5RMJUOBGKYK2JX52P2HD' # your Foursquare Secret
VERSION = '20200521'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3TTX1OYJ5M1TNYZNOFUQIYCQWE4OL2J2ZKX1LQNSJEUD2SRA
CLIENT_SECRET:U5TUV4RLEPPEFPEIO32K3CB5LPLW5RMJUOBGKYK2JX52P2HD


In [8]:
address = 'Dubai'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.0659637 55.1713403


In [18]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Dubai',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df.maplocationlatitude, df.maplocationlongitude, df.typeofevent):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map